# Algorithm prototyping

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import shapely as sh
import shapely.geometry as shg
import shapely.ops as sho

In [2]:
from pyproj import CRS, Transformer

ImportError: cannot import name 'CRS' from 'pyproj' (C:\ProgramData\Anaconda3\lib\site-packages\pyproj\__init__.py)

In [ ]:
%matplotlib inline

## Helper functions

In [ ]:
crs_4326 = CRS.from_epsg(4326) # WGS84
crs_25832 = CRS.from_epsg(25832) # ETRS89 / UTM zone 32N
to_wgs_84_transformer = Transformer.from_crs(crs_25832, crs_4326)
to_utm_transformer = Transformer.from_crs(crs_4326, crs_25832)

In [ ]:
df = pd.read_csv("data\extracted\dom1l-fp_32349_5660_1_nw.csv")

In [ ]:
df = df.sample(10000)

In [ ]:
df = gpd.GeoDataFrame(df)

In [ ]:
df.head()

In [ ]:
# creating a geometry column 
geometry = [sh.geometry.Point(xy) for xy in zip(df['lon'], df['lat'])]

# Coordinate reference system : WGS84
crs = {'init': 'epsg:4326'}

In [ ]:
# Creating a Geographic data frame 
gdf = gpd.GeoDataFrame(df[["x", "y", "z"]], crs=crs, geometry=geometry)

In [ ]:
gdf.head()

In [ ]:
lat_q, lon_q = 51.078534, 6.844748
query_point = shg.Point(lat_q, lon_q)

In [ ]:
# filter to 200m around query point first
R = 50
query_point_xy = shg.Point(to_utm_transformer.transform(lon_q, lat_q))
query_region_xy = query_point_xy.buffer(R, resolution=5)
print(query_region_xy)
query_region_wgs84 = sho.transform(to_wgs_84_transformer.transform, query_region_xy)

In [ ]:
print(query_region_wgs84)

In [ ]:
gdf_query = gdf[gdf.intersects(query_region_wgs84)]

In [ ]:
len(gdf_query), len(gdf)